In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import seaborn as sns
import feature_selection as fs
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
import prep_utils as pu
import numpy as np
from scipy.stats import pointbiserialr

In [2]:
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()

[df_awards, df_coaches, df_players_teams, df_players, df_series_post, df_teams_post, df_teams] = pu.db_to_pandas(db)

In [3]:
df_new_teams = pu.prepare_teams(df_teams,df_teams_post,1)
df_new_teams = fs.fs_teams(df_new_teams)

df_team_results = df_new_teams[["year","tmID","playoff","rank","team_playoffs_count"]]
fs.bisserial_corr(df_new_teams)
#print(df_new_teams.to_string())
#fs.correlation_matrix(df_new_teams)

Dropping divID in Teams...
PO_Winrate: 30.90% correlation
Winrate: 24.60% correlation
team_playoffs_count: 18.41% correlation
o_asts: 14.32% correlation
o_blk: 14.16% correlation
o_pts: 12.76% correlation
rank: 12.22% correlation
o_fg_pct: 10.48% correlation
d_pf: 10.04% correlation
o_ft_pct: 9.92% correlation
d_ft_pct: 9.60% correlation
o_3p_pct: 9.41% correlation
d_pts: 8.94% correlation
min: 8.79% correlation
o_to: 8.62% correlation
o_pf: 8.26% correlation
o_stl: 8.22% correlation
o_reb_pct: 8.17% correlation
d_asts: 8.03% correlation
d_to: 7.70% correlation
d_reb_pct: 6.96% correlation
d_stl: 6.81% correlation
d_fg_pct: 6.59% correlation
d_3p_pct: 5.29% correlation
d_blk: 3.95% correlation


In [4]:
df_new_coaches = pu.prepare_coaches(df_coaches, df_awards,1)
df_new_coaches = pu.group_coaches(df_new_coaches)
df_new_coaches.drop("coachID", axis = 1, inplace = True)

df_final_coaches = df_new_coaches.copy()
df_final_coaches.columns = df_final_coaches.columns.str.lower()

Dropping Attribute lgID in Coaches...
Creating attribute coach previous regular season win ratio...
Creating attribute coach playoffs win ratio...
Creating attribute coach playoffs count...
Creating attribute coach awards count...
Dropping attribute post_wins..
Dropping attribute post_losses..
Dropping attribute won..
Dropping attribute lost..

Coaches Null Verification:
year                    0
tmID                    0
coachID                 0
coach_reg_wr            0
coach_po_wr             0
coach_playoffs_count    0
coach_awards            0
dtype: int64


In [5]:
df_new_players_teams = pu.prepare_player_teams(df_players_teams,df_awards,1)
df_players = df_new_players_teams.copy()
df_players = fs.fs_players(df_players)
df_players = df_players[df_players['year'] != 1]

df_team_results.columns = df_team_results.columns.str.lower()
merged_data = pd.merge(df_players, df_team_results, on=['tmid', 'year'], how='left')

fs.bisserial_corr(merged_data)

# Dropping features with < 10% correlation
merged_data.drop(columns=["gp","total_orebounds_pct","total_ft_pct","total_three_pct","total_fg_pct"], axis= 1, inplace= True)


Dropping Attribute lgID in Players_Teams...
total_turnovers: 36.84% correlation
total_assists: 36.12% correlation
total_steals: 35.11% correlation
total_points: 35.03% correlation
total_minutes: 34.87% correlation
total_blocks: 32.84% correlation
total_pf: 32.53% correlation
player_awards: 27.79% correlation
postgp: 24.68% correlation
team_playoffs_count: 19.10% correlation
rank: 18.01% correlation
total_dq: 16.85% correlation
gp: 7.77% correlation
total_orebounds_pct: 6.87% correlation
total_ft_pct: 3.16% correlation
total_three_pct: 2.80% correlation
total_fg_pct: 0.44% correlation


In [6]:
from sklearn.preprocessing import MinMaxScaler
label_encoder = LabelEncoder()
merged_data['tmid'] = label_encoder.fit_transform(merged_data['tmid'])

x = merged_data.drop('playoff', axis=1)
y = merged_data['playoff']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()

# Define the hyperparameter grid to search
param_grid = {
    'C': np.logspace(-3, 3, 7),  # Regularization parameter
    'penalty': ['l1', 'l2'],       # Regularization type
    'solver': ['liblinear', 'saga'],  # Algorithm to use for optimization
    'max_iter': [100, 200, 300],  # Maximum number of iterations
    'class_weight': [None, 'balanced']  # Weighting of classes
}

# Create the GridSearchCV object
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

# Print the best parameters and corresponding accuracy
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

# Get the best model
best_logreg = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_logreg.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: ", accuracy)



/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_mode

Best Parameters:  {'C': 0.001, 'class_weight': 'balanced', 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}
Best Accuracy:  0.6100000000000001
Test Accuracy:  0.6153846153846154


/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/oliveira/.local/lib/python3.10/site-packages/sklearn/linear_mode